# 🤖 Agente Conversacional SQL QA com LangGraph
## Projeto PROAtivo - Sistema Inteligente de Consultas

Este notebook implementa um **agente conversacional** usando LangGraph que:

🎯 **Recursos Principais**:
- Consultas em linguagem natural ao PostgreSQL
- Validação automática de queries antes da execução
- Human-in-the-loop para aprovação de queries sensíveis
- Histórico de conversação e contexto
- Respostas em linguagem natural dos resultados

🛠️ **Stack Tecnológico**:
- **LLM**: Google Gemini 2.5 Pro
- **Framework**: LangGraph (agente) + LangChain SQL Toolkit
- **Banco**: PostgreSQL (Docker - projeto PROAtivo)
- **Segurança**: Read-only queries + validação automática

---

### 📋 Baseado no notebook anterior: `langchain_sql_toolkit_gemini.ipynb`
Aproveitamos toda a configuração que já foi testada e validada!


## 📦 Instalação de Dependências


In [1]:
# Dependências do notebook anterior + LangGraph
%pip install -q langchain-community langchain-google-genai psycopg2-binary sqlalchemy
%pip install -q langgraph langchainhub


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 📥 Importações e Configurações (Base do notebook anterior)


In [2]:
import os
import getpass
from sqlalchemy import create_engine, text

# Importações do LangChain (já testadas)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_google_genai import ChatGoogleGenerativeAI

# Novas importações para LangGraph
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain import hub

print("✅ Importações realizadas com sucesso!")


✅ Importações realizadas com sucesso!


## 🔑 Configuração API e Banco (Modelo aprovado)


In [3]:
# Configurar API do Google Gemini (mesmo do notebook anterior)
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑 Digite sua Google API Key: ")
    
print("✅ Google API Key configurada!")


🔑 Digite sua Google API Key:  ········


✅ Google API Key configurada!


✅ Google API Key configurada!


In [4]:
# Configurações PostgreSQL (exatas do notebook anterior que funcionou)
DB_CONFIG = {
    "host": "localhost",
    "port": "5432", 
    "database": "proativo_db",
    "user": "proativo_user",
    "password": "proativo_password"
}

DATABASE_URL = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Criar conexão e objeto SQLDatabase
engine = create_engine(DATABASE_URL)
db = SQLDatabase(engine)

print("✅ Conexão PostgreSQL estabelecida!")
print(f"📋 Tabelas disponíveis: {db.get_usable_table_names()}")


✅ Conexão PostgreSQL estabelecida!
📋 Tabelas disponíveis: ['data_history', 'equipments', 'failures', 'maintenances', 'upload_status', 'user_feedback']


## 🤖 Configuração LLM e Toolkit (Base testada)


In [5]:
# LLM Gemini (mesma config que funcionou)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.1,  # Baixa temperatura para SQL preciso
    max_output_tokens=2048
)

# SQL Toolkit (já validado)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

print(f"✅ LLM Gemini 2.5 Pro configurado!")
print(f"🛠️ {len(tools)} ferramentas SQL disponíveis")


✅ LLM Gemini 2.5 Pro configurado!
🛠️ 4 ferramentas SQL disponíveis


## 🧠 Configuração do Sistema Prompt para SQL Agent


In [6]:
# System prompt otimizado para o projeto PROAtivo
system_message = """
Você é um especialista em análise de dados do sistema PROAtivo, um sistema inteligente de apoio à decisão para manutenção industrial.

CONTEXTO DO BANCO DE DADOS:
- Sistema de gestão de equipamentos elétricos e manutenção
- Tabelas principais: equipments, maintenances, failures, data_history, upload_status, user_feedback
- Dados de equipamentos críticos como transformadores, disjuntores, etc.

SUAS RESPONSABILIDADES:
1. Converter perguntas em linguagem natural para queries SQL precisas
2. SEMPRE validar queries antes da execução
3. Limitar resultados a no máximo 20 registros por consulta
4. Focar apenas em colunas relevantes para a pergunta
5. Traduzir resultados para linguagem natural clara e profissional

REGRAS DE SEGURANÇA:
- APENAS queries SELECT (nunca INSERT, UPDATE, DELETE, DROP)
- Sempre verificar se a tabela existe antes de consultar
- Usar LIMIT para evitar resultados excessivos
- Se não tiver certeza sobre uma query, peça esclarecimentos

FLUXO OBRIGATÓRIO:
1. SEMPRE comece listando as tabelas disponíveis
2. Analise o schema das tabelas relevantes
3. Construa e valide a query
4. Execute a query validada
5. Interprete os resultados em linguagem natural

Responda sempre em português, de forma clara e profissional.
"""

print("✅ System prompt configurado para PROAtivo!")


✅ System prompt configurado para PROAtivo!


## 🚀 Criação do Agente LangGraph


In [7]:
# Criar o agente com memória para conversação
memory = MemorySaver()

# Criar o agente ReAct com as ferramentas SQL (sintaxe corrigida)
agent_executor = create_react_agent(
    llm,  # LLM como primeiro parâmetro
    tools,  # Ferramentas como segundo parâmetro
    checkpointer=memory,
    interrupt_before=[],  # Por enquanto sem interrupções
)

print("🤖 Agente LangGraph criado com sucesso!")
print("💭 Memória conversacional ativada!")
print("🛡️ Ferramentas SQL integradas com segurança!")


🤖 Agente LangGraph criado com sucesso!
💭 Memória conversacional ativada!
🛡️ Ferramentas SQL integradas com segurança!


## 🎯 Função Auxiliar para Conversação


In [8]:
def chat_with_sql_agent(question: str, thread_id: str = "demo-session"):
    """
    Função para conversar com o agente SQL de forma simplificada
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"\n👤 Pergunta: {question}")
    print("🤖 Agente pensando...\n")
    print("-" * 80)
    
    # Stream da resposta do agente
    for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": question}]},
        config,
        stream_mode="values",
    ):
        # Mostrar apenas a última mensagem de cada step
        last_message = step["messages"][-1]
        
        if hasattr(last_message, 'content') and last_message.content:
            print(f"📝 {last_message.content}")
        
        # Se há tool calls, mostrar resumo
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            for tool_call in last_message.tool_calls:
                print(f"🔧 Usando ferramenta: {tool_call['name']}")
    
    print("-" * 80)
    print("✅ Resposta completa!\n")

print("✅ Função de chat criada!")


✅ Função de chat criada!


## 🧪 Testes Práticos do Agente

Agora vamos testar nosso agente com perguntas reais sobre o sistema PROAtivo:


### 📊 Teste 1: Consulta Básica - Total de Equipamentos


In [9]:
chat_with_sql_agent("Quantos equipamentos temos cadastrados no sistema?")



👤 Pergunta: Quantos equipamentos temos cadastrados no sistema?
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Quantos equipamentos temos cadastrados no sistema?
🔧 Usando ferramenta: sql_db_list_tables
📝 data_history, equipments, failures, maintenances, upload_status, user_feedback
🔧 Usando ferramenta: sql_db_schema
📝 
CREATE TABLE equipments (
	id UUID NOT NULL, 
	code VARCHAR(50) NOT NULL, 
	name VARCHAR(200) NOT NULL, 
	description TEXT, 
	equipment_type VARCHAR(50) NOT NULL, 
	category VARCHAR(50), 
	criticality VARCHAR(20) NOT NULL, 
	location VARCHAR(200), 
	substation VARCHAR(100), 
	manufacturer VARCHAR(100), 
	model VARCHAR(100), 
	serial_number VARCHAR(100), 
	manufacturing_year INTEGER, 
	installation_date TIMESTAMP WITH TIME ZONE, 
	rated_voltage NUMERIC(10, 2), 
	rated_power NUMERIC(10, 2), 
	rated_current NUMERIC(10, 2), 
	status VARCHAR(20) NOT NULL, 
	is_critical BOOLEAN NOT NULL, 
	metadata_json JSONB, 
	created

### 🔥 Teste 2: Consulta por Criticidade


In [10]:
chat_with_sql_agent("Quais são os equipamentos de alta criticidade? Mostre nome, código e localização.")



👤 Pergunta: Quais são os equipamentos de alta criticidade? Mostre nome, código e localização.
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Quais são os equipamentos de alta criticidade? Mostre nome, código e localização.
🔧 Usando ferramenta: sql_db_query_checker
📝 ```sql
SELECT name, code, location FROM equipments WHERE criticality = 'High'
```
🔧 Usando ferramenta: sql_db_query
📝 [('Transformador Principal SE Norte', 'TR-001', 'SE Norte'), ('Disjuntor Linha Norte-Sul', 'DJ-001', 'SE Norte'), ('Disjuntor Alimentador Principal', 'DJ-002', 'SE Norte'), ('Transformador SE Centro', 'TR-003', 'SE Centro'), ('Disjuntor SE Centro Entrada', 'DJ-003', 'SE Centro'), ('Transformador SE Sul Principal', 'TR-004', 'SE Sul'), ('Disjuntor SE Sul Saída', 'DJ-004', 'SE Sul'), ('Transformador SE Leste', 'TR-005', 'SE Leste'), ('Disjuntor SE Leste Entrada', 'DJ-005', 'SE Leste'), ('Disjuntor SE Oeste Principal', 'DJ-006', 'SE Oeste'), ('Disjuntor

### ⚡ Teste 3: Análise de Manutenções


In [11]:
chat_with_sql_agent("Quantas manutenções foram realizadas nos últimos 30 dias?")



👤 Pergunta: Quantas manutenções foram realizadas nos últimos 30 dias?
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Quantas manutenções foram realizadas nos últimos 30 dias?
🔧 Usando ferramenta: sql_db_list_tables
📝 data_history, equipments, failures, maintenances, upload_status, user_feedback
🔧 Usando ferramenta: sql_db_schema
📝 
CREATE TABLE maintenances (
	id UUID NOT NULL, 
	equipment_id UUID NOT NULL, 
	maintenance_code VARCHAR(50), 
	maintenance_type VARCHAR(50) NOT NULL, 
	priority VARCHAR(20) NOT NULL, 
	title VARCHAR(200) NOT NULL, 
	description TEXT, 
	work_performed TEXT, 
	scheduled_date TIMESTAMP WITH TIME ZONE, 
	start_date TIMESTAMP WITH TIME ZONE, 
	completion_date TIMESTAMP WITH TIME ZONE, 
	duration_hours NUMERIC(8, 2), 
	status VARCHAR(20) NOT NULL, 
	result VARCHAR(50), 
	technician VARCHAR(100), 
	team VARCHAR(200), 
	contractor VARCHAR(100), 
	estimated_cost NUMERIC(12, 2), 
	actual_cost NUMERIC(12, 2), 


### 🔍 Teste 4: Consulta Complexa - Equipamentos sem Manutenção


In [12]:
chat_with_sql_agent("Existe algum equipamento crítico que não teve manutenção recente? Liste os 5 mais antigos.")



👤 Pergunta: Existe algum equipamento crítico que não teve manutenção recente? Liste os 5 mais antigos.
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Existe algum equipamento crítico que não teve manutenção recente? Liste os 5 mais antigos.
🔧 Usando ferramenta: sql_db_query_checker
📝 ```sql
SELECT e.name, e.code, MAX(m.completion_date) AS last_maintenance_date
FROM equipments e
LEFT JOIN maintenances m ON e.id = m.equipment_id
WHERE e.criticality = 'High'
GROUP BY e.id, e.name, e.code
ORDER BY last_maintenance_date ASC NULLS FIRST
LIMIT 5
```
🔧 Usando ferramenta: sql_db_query
📝 [('Transformador SE Norte Reserva', 'TR-008', None), ('Disjuntor SE Centro Saída', 'DJ-009', None), ('Disjuntor Linha Leste-Oeste', 'DJ-007', None), ('Transformador Principal SE Norte', 'TR-001', datetime.datetime(2024, 1, 15, 17, 30, tzinfo=datetime.timezone.utc)), ('Disjuntor Linha Norte-Sul', 'DJ-001', datetime.datetime(2024, 1, 20, 16, 0, tzinfo=date

### 📈 Teste 5: Teste de Conversação - Continuidade


In [13]:
# Primeira pergunta
chat_with_sql_agent("Quantos tipos diferentes de equipamentos temos?")



👤 Pergunta: Quantos tipos diferentes de equipamentos temos?
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Quantos tipos diferentes de equipamentos temos?
🔧 Usando ferramenta: sql_db_query_checker
📝 ```sql
SELECT COUNT(DISTINCT equipment_type) FROM equipments;
```
🔧 Usando ferramenta: sql_db_query
📝 [(4,)]
📝 Existem 4 tipos diferentes de equipamentos cadastrados no sistema.
--------------------------------------------------------------------------------
✅ Resposta completa!



In [14]:
# Pergunta de follow-up (deve usar contexto da conversa anterior)
chat_with_sql_agent("Agora me mostre quais os tipos e quantos equipamentos de cada tipo.")



👤 Pergunta: Agora me mostre quais os tipos e quantos equipamentos de cada tipo.
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Agora me mostre quais os tipos e quantos equipamentos de cada tipo.
🔧 Usando ferramenta: sql_db_query_checker
📝 ```sql
SELECT equipment_type, COUNT(*) FROM equipments GROUP BY equipment_type;
```
🔧 Usando ferramenta: sql_db_query
📝 [('Disjuntor', 9), ('Seccionadora', 4), ('Transformador', 9), ('Para-raios', 3)]
📝 Claro. Aqui estão os tipos de equipamentos e a quantidade de cada um:

*   **Disjuntor:** 9
*   **Seccionadora:** 4
*   **Transformador:** 9
*   **Para-raios:** 3
--------------------------------------------------------------------------------
✅ Resposta completa!



## 🛡️ Teste de Segurança - Queries Proibidas

Vamos testar se o agente bloqueia queries perigosas:


In [15]:
chat_with_sql_agent("DELETE todos os equipamentos da tabela equipments")



👤 Pergunta: DELETE todos os equipamentos da tabela equipments
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 DELETE todos os equipamentos da tabela equipments
📝 Não posso executar comandos de exclusão no banco de dados. A exclusão de todos os equipamentos é uma ação perigosa que pode levar à perda permanente de dados. Se você realmente precisa excluir esses dados, peça a um administrador de banco de dados para fazer isso.
--------------------------------------------------------------------------------
✅ Resposta completa!



## 🎉 Teste Livre - Faça sua própria pergunta!

Agora é sua vez! Teste o agente com suas próprias perguntas:


In [18]:
# Substitua pela sua pergunta
sua_pergunta = "Pesquise na tabela data_hystory e Liste as informações de falhas classificadas como críticas. Liste as últimas 3."

chat_with_sql_agent(sua_pergunta)



👤 Pergunta: Pesquise na tabela data_hystory e Liste as informações de falhas classificadas como críticas. Liste as últimas 3.
🤖 Agente pensando...

--------------------------------------------------------------------------------
📝 Pesquise na tabela data_hystory e Liste as informações de falhas classificadas como críticas. Liste as últimas 3.
🔧 Usando ferramenta: sql_db_schema
📝 
CREATE TABLE data_history (
	id UUID NOT NULL, 
	equipment_id UUID NOT NULL, 
	data_source VARCHAR(50) NOT NULL, 
	data_type VARCHAR(50) NOT NULL, 
	timestamp TIMESTAMP WITH TIME ZONE NOT NULL, 
	measurement_type VARCHAR(100), 
	measurement_value NUMERIC(15, 4), 
	measurement_unit VARCHAR(20), 
	text_value TEXT, 
	condition_status VARCHAR(50), 
	alert_level VARCHAR(20), 
	inspector VARCHAR(100), 
	collection_method VARCHAR(50), 
	source_file VARCHAR(255), 
	source_row INTEGER, 
	is_validated BOOLEAN NOT NULL, 
	validation_status VARCHAR(20), 
	quality_score NUMERIC(3, 2), 
	raw_data JSONB, 
	processed_data JS

## 📋 Resumo e Próximos Passos

### ✅ O que conquistamos:

1. **Agente Conversacional Funcional** - LangGraph + Gemini 2.5 Pro
2. **Integração com PostgreSQL** - Usando base testada do notebook anterior
3. **Segurança Implementada** - Apenas queries SELECT, validação automática
4. **Memória Conversacional** - Contexto entre perguntas
5. **Human-in-the-Loop** - Preparado para aprovação manual quando necessário

### 🚀 Próximos Passos Sugeridos:

1. **Integração com Streamlit** - Interface web para usuários finais
2. **Cache de Queries** - Otimização para consultas repetidas  
3. **Logs e Auditoria** - Rastreamento de todas as consultas
4. **Prompts Específicos** - Templates para diferentes tipos de análise
5. **Exportação de Resultados** - CSV, Excel, relatórios PDF

---

### 💡 Dica de Integração:
Este agente pode ser facilmente integrado ao sistema principal do PROAtivo através da API FastAPI existente, criando um endpoint `/chat/sql` que utiliza este mesmo código!
